In [1]:
!pip install psycopg2

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2

### Extract CSVs into DataFrames

In [3]:
cost_file = "./Resources/movehubcostofliving.csv"
cost_df = pd.read_csv(cost_file)
cost_df.head()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income
0,Lausanne,3.15,12.59,8.40,1.32,1714.00,4266.11
1,Zurich,3.28,12.59,8.40,1.31,2378.61,4197.55
2,Geneva,2.80,12.94,10.49,1.28,2607.95,3917.72
3,Basel,3.50,11.89,7.35,1.25,1649.29,3847.76
4,Perth,2.87,11.43,10.08,0.97,2083.14,3358.55


In [4]:
cities_file = "./Resources/cities.csv"
cities_df = pd.read_csv(cities_file)
cities_df.head()

,City,Country
0,Oakland,United States
1,Oakville,Canada
2,Oaxaca de Juárez,Mexico
3,Oberhausen,Germany
4,Obihiro,Japan


In [5]:
quality_file = "./Resources/movehubqualityoflife.csv"
quality_df = pd.read_csv(quality_file)
quality_df.head()

,City,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
0,Caracas,65.18,11.25,44.44,83.45,8.61,85.70
1,Johannesburg,84.08,53.99,59.98,47.39,51.26,83.93
2,Fortaleza,80.17,52.28,45.46,66.32,36.68,78.65
3,Saint Louis,85.25,80.40,77.29,31.33,87.51,78.13
4,Mexico City,75.07,24.28,61.76,18.95,27.91,77.86


In [6]:
m_pfile = "./Project_ETL/Output/movehub_and_populations.csv"
m_p = pd.read_csv(m_pfile)
m_p.head()

,City,Country,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Population,Prev,Growth
0,Geneva,Switzerland,2.80,12.94,10.49,1.28,2607.95,3917.72,83.27,61.22,74.88,29.43,82.76,54.36,613373.0,606436.0,0.0114
1,Basel,Switzerland,3.50,11.89,7.35,1.25,1649.29,3847.76,84.20,78.17,79.74,59.18,88.27,28.12,559027.0,555469.0,0.0064
2,Perth,Australia,2.87,11.43,10.08,0.97,2083.14,3358.55,95.38,62.11,80.56,23.53,74.62,50.01,2041959.0,2016415.0,0.0127
3,Nashville,United States,3.84,12.00,13.50,0.65,2257.14,3089.75,80.61,80.30,60.30,0.00,80.50,25.50,673167.0,671110.0,0.0031
4,Adelaide,Australia,2.49,11.42,10.08,0.95,1382.26,2911.69,87.29,72.03,56.25,12.01,91.54,41.32,1336403.0,1328119.0,0.0062


### Transform Cost DataFrame
* Is there a relationship between average rent cost and average disposable income accross cities around the globe vs quality of life ranking?
* Do people pay higher rent in order to live in cities with higher quality of life rankings?
* If they do, is it because they have higher averages in disposable income?
* Or, because the average disposable income is higher is it used on things like cappuccinos and wine?


In [7]:
cost_df.head()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income
0,Lausanne,3.15,12.59,8.40,1.32,1714.00,4266.11
1,Zurich,3.28,12.59,8.40,1.31,2378.61,4197.55
2,Geneva,2.80,12.94,10.49,1.28,2607.95,3917.72
3,Basel,3.50,11.89,7.35,1.25,1649.29,3847.76
4,Perth,2.87,11.43,10.08,0.97,2083.14,3358.55


In [8]:
#Check for duplicates
cost_df[cost_df.duplicated()]

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income


In [9]:
#Check for null values
cost_df.isna().any()

City                     False
Cappuccino               False
Cinema                   False
Wine                     False
Gasoline                 False
Avg Rent                 False
Avg Disposable Income    False
dtype: bool

In [10]:
# Create filtered dataframe from specific columns
cost_cols = ["City", "Avg Rent", "Avg Disposable Income"]
cost_transformed = cost_df[cost_cols].copy()

# Rename the column headers
cost_transformed = cost_transformed.rename(columns={
    "Avg Rent": "Average_Rent_Cost",
    "Avg Disposable Income": "Average_Disposable_Income"
})

# Set the index to City
cost_transformed.set_index("City", inplace=True)
cost_transformed.head()

,Average_Rent_Cost,Average_Disposable_Income
City,,
Lausanne,1714.00,4266.11
Zurich,2378.61,4197.55
Geneva,2607.95,3917.72
Basel,1649.29,3847.76
Perth,2083.14,3358.55


In [11]:
# Sort by Avg rent cost
src = cost_transformed.sort_values('Average_Rent_Cost', ascending=False)
src.head()

,Average_Rent_Cost,Average_Disposable_Income
City,,
Hong Kong,5052.31,2210.39
New York,3268.84,2530.09
Singapore,3164.42,1582.21
Sydney,2788.71,2755.12
Geneva,2607.95,3917.72


In [12]:
# Sort by Avg Disposable Income
sdi = cost_transformed.sort_values('Average_Disposable_Income', ascending=False)
sdi.head()

,Average_Rent_Cost,Average_Disposable_Income
City,,
Lausanne,1714.00,4266.11
Zurich,2378.61,4197.55
Geneva,2607.95,3917.72
Basel,1649.29,3847.76
Perth,2083.14,3358.55


### Transform Quality DataFrame
* Is there a relationship between average rent cost and average disposable income accross cities around the globe?
* Do people pay higher rent in order to live in cities with higher quality of life rankings?
* If they do, is it because they have higher averages in disposable income?
* Or, because the average disposable income is higher is it used on things like cappuccinos and wine?

In [13]:
quality_df.head()

,City,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
0,Caracas,65.18,11.25,44.44,83.45,8.61,85.70
1,Johannesburg,84.08,53.99,59.98,47.39,51.26,83.93
2,Fortaleza,80.17,52.28,45.46,66.32,36.68,78.65
3,Saint Louis,85.25,80.40,77.29,31.33,87.51,78.13
4,Mexico City,75.07,24.28,61.76,18.95,27.91,77.86


In [14]:
#Check for duplicates
quality_df[quality_df.duplicated()]

,City,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating


In [15]:
#Check for null values
quality_df.isna().any()

City               False
Movehub Rating     False
Purchase Power     False
Health Care        False
Pollution          False
Quality of Life    False
Crime Rating       False
dtype: bool

In [16]:
quality_cols = ["City", "Quality of Life"]
quality_transformed = quality_df[quality_cols].copy()

# Rename the columns
quality_transformed = quality_transformed.rename(columns={
    "Quality of Life": "Quality of Life Rating"
})

# Set the index to cities
# Set the index to City
quality_transformed.set_index("City", inplace=True)
quality_transformed.head()

,Quality of Life Rating
City,
Caracas,8.61
Johannesburg,51.26
Fortaleza,36.68
Saint Louis,87.51
Mexico City,27.91


In [17]:
# Sort by Quality of life rating
sq = quality_transformed.sort_values('Quality of Life Rating', ascending=False)
sq.head()

,Quality of Life Rating
City,
Zurich,97.91
Canberra,93.05
Adelaide,91.54
Berlin,91.17
Aachen,90.52


In [18]:
# Merge quality and cost dataframes together
merge_cq = pd.merge(sq, cost_transformed, on='City', how='outer')
merge_cq.head()

,Quality of Life Rating,Average_Rent_Cost,Average_Disposable_Income
City,,,
Zurich,97.91,2378.61,4197.55
Canberra,93.05,1984.74,3023.91
Adelaide,91.54,1382.26,2911.69
Berlin,91.17,916.42,1772.57
Aachen,90.52,767.23,1619.72


In [19]:
#Check for null values
merge_cq.isna().any()

Quality of Life Rating       False
Average_Rent_Cost            False
Average_Disposable_Income    False
dtype: bool

### Create database connection

In [20]:
# Connect to the database
connection_string = "postgres:25162516@localhost:5432/ETL_db"
engine = create_engine(f'postgresql://{connection_string}')

In [21]:
# confirm tables
engine.table_names()

['QOL', 'merged_cq', 'COL', 'm_pc']

### Load DataFrames into Database


In [22]:
sq.to_sql(name='QOL', con=engine, if_exists='append', index=True)

In [23]:
merge_cq.to_sql(name='merged_cq', con=engine, if_exists='append', index=True)

In [24]:
cost_transformed.to_sql(name='COL', con=engine, if_exists='append', index=True)

In [25]:
m_p.to_sql(name='m_pc', con=engine, if_exists='append', index=True)

### Confirm data has been added 

In [26]:
pd.read_sql_query('select * from m_pc', con=engine).head()

,index,Unnamed: 0,City,Country,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Population,Prev,Growth
0,0,1.0,Geneva,Switzerland,2.80,12.94,10.49,1.28,2607.95,3917.72,83.27,61.22,74.88,29.43,82.76,54.36,613373.0,606436.0,0.0114
1,1,2.0,Basel,Switzerland,3.50,11.89,7.35,1.25,1649.29,3847.76,84.20,78.17,79.74,59.18,88.27,28.12,559027.0,555469.0,0.0064
2,2,3.0,Perth,Australia,2.87,11.43,10.08,0.97,2083.14,3358.55,95.38,62.11,80.56,23.53,74.62,50.01,2041959.0,2016415.0,0.0127
3,3,4.0,Nashville,United States,3.84,12.00,13.50,0.65,2257.14,3089.75,80.61,80.30,60.30,0.00,80.50,25.50,673167.0,671110.0,0.0031
4,4,9.0,Adelaide,Australia,2.49,11.42,10.08,0.95,1382.26,2911.69,87.29,72.03,56.25,12.01,91.54,41.32,1336403.0,1328119.0,0.0062


In [27]:
pd.read_sql_query('select * from merged_cq', con=engine).head()

,City,Quality of Life Rating,Average_Rent_Cost,Average_Disposable_Income
0,Zurich,97.91,2378.61,4197.55
1,Canberra,93.05,1984.74,3023.91
2,Adelaide,91.54,1382.26,2911.69
3,Berlin,91.17,916.42,1772.57
4,Aachen,90.52,767.23,1619.72
